In [1]:
#Import Packages
import pandas as pd
import numpy as np
import os
import itertools
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import re
import matplotlib.pyplot as plt
from PIL import Image
import datetime
import matplotlib
import matplotlib.dates as mdates
from sklearn.linear_model import LinearRegression
import string
import statsmodels.api as sm
from herbie import Herbie
import pickle
import cartopy.crs as ccrs
import cartopy.io.img_tiles as cimgt
import pysolar.solar as solar
from geographiclib.geodesic import Geodesic
import xarray as xr
import pytz
import simplekml
from pylr2 import regress2
import cartopy
import sklearn
import sys
sys.path.append('..')
import funcs.ac_funcs as ac
import funcs.plotting as plotting

pd.options.mode.chained_assignment = None

%load_ext autoreload
%autoreload 2

# Aaron Vs Elaine 
UA Summer 2023 Retrievals.\
Aaron retrievals using EGI on UoU CHPC, no airmass correction\
Elain retrievals at Harvard, with airmass correction

In [ ]:
#Define parameters
base_project_dir = '/uufs/chpc.utah.edu/common/home/u0890904/LAIR_1/'
inst_ids = ['ha','ua']
flag = 0
resample = 'T'
timezone = 'US/Mountain' 
specs = ['xch4(ppm)','xco2(ppm)','xco(ppb)']

In [ ]:
#Load the EM27 Data
aaron_folder = "/uufs/chpc.utah.edu/common/home/u0890904/LAIR_1/Data/EM27_oof/summer_2023/aaron_retrievals/ha/"
elaine_folder = "/uufs/chpc.utah.edu/common/home/u0890904/LAIR_1/Data/EM27_oof/summer_2023/elaine_retrievals/ha/"

dt1_str = '2023-07-08 11:00:00'
dt2_str = '2023-07-11 23:59:59' 
tz = 'UTC'

aaron_oof_manage = ac.oof_manager(aaron_folder,tz)
aaron_em27_df = aaron_oof_manage.load_oof_df_inrange(dt1_str,dt2_str,True)

elaine_oof_manage = ac.oof_manager(elaine_folder,tz)
elaine_em27_df = elaine_oof_manage.load_oof_df_inrange(dt1_str,dt2_str,True)


In [ ]:
species = ['xch4(ppm)','xco2(ppm)','xco(ppb)']
fig = make_subplots(rows=3,cols=1)
row = 1
for spec in species:
    if row == 1:
        showlegend = True
    else:
        showlegend = False
    fig.add_trace(go.Scatter(
        x = aaron_em27_df.index,
        y = aaron_em27_df[spec],
        mode = 'markers',
        marker = {'color' : 'grey'},
        name = 'aaron',
        showlegend = showlegend
    ), row = row , col = 1)
    fig.add_trace(go.Scatter(
        x = elaine_em27_df.index,
        y = elaine_em27_df[spec],
        mode = 'markers',
        marker = {'color' : 'red'},
        name = 'elaine',
        showlegend = showlegend
    ), row = row , col = 1)
    fig.update_yaxes(title_text=f'{spec}',row=row,col=1)
    row += 1

fig.update_layout(
    height=600,
    margin = go.layout.Margin(t=1,b=1)
)

# Airmass Correction Comparison
Both retrieved by Elaine at Harvard\
One with Nasrin's airmass correction\
One without

In [ ]:
#Loading the full datasets can take a while
dt1_str = '2022-05-23 00:00:00'
dt2_str =  '2022-12-01 00:00:00'
tz = 'UTC'

em27_data_folder_v1 = "/uufs/chpc.utah.edu/common/home/u0890904/LAIR_1/Data/EM27_oof/SLC_EM27_ha_2022_2023_oof_v1/"
em27_data_folder_v2 = "/uufs/chpc.utah.edu/common/home/u0890904/LAIR_1/Data/EM27_oof/SLC_EM27_ha_2022_2023_oof_v2_nasrin_correct/"

oof_manage_v1 = ac.oof_manager(em27_data_folder_v1,tz)
em27_df_v1 = oof_manage_v1.load_oof_df_inrange(dt1_str,dt2_str,False)

oof_manage_v2 = ac.oof_manager(em27_data_folder_v2,tz)
em27_df_v2 = oof_manage_v2.load_oof_df_inrange(dt1_str,dt2_str,False)

## Flag Differences

We examine both V1 and V2 datasets for differences between flags. V2 has many more rows of flag=0 (good) data. All of the rows that are flag=0 in V2 and are NOT flag=0 in V1 are flag=99 in V1. 

In [ ]:
flag0df_v1 = em27_df_v1.loc[em27_df_v1['flag']==0]
flag0df_v2 = em27_df_v2.loc[em27_df_v2['flag']==0]
diff_flag_idxs = flag0df_v2.index.difference(flag0df_v1.index)

diff_flags_df_v1 = em27_df_v1.loc[diff_flag_idxs]
diff_flags_df_v2 = em27_df_v2.loc[diff_flag_idxs]

print(f"V1 has {len(flag0df_v1)} rows of flag=0 data")
print(f"V2 has {len(flag0df_v2)} rows of flag=0 data")
print(f"V2 has {len(diff_flag_idxs)} more flag=0 data")

In [ ]:
diff_flags_df_v1.groupby('flag').count()

In [ ]:
subtract_diff = flag0df_v1.drop(['spectrum'],axis = 1).subtract(flag0df_v2.drop(['spectrum'],axis=1))

In [ ]:
labsize = 18

plotdf_v1 = em27_df_v1.loc[(em27_df_v1.index>'2022-07-20')&
                       (em27_df_v1.index<'2022-07-23')]
plotdf_v2 = em27_df_v2.loc[(em27_df_v2.index>'2022-07-20')&
                       (em27_df_v2.index<'2022-07-23')]

fig,ax = plt.subplots(figsize = (20,10))
ax.scatter(plotdf_v1.index[10:],plotdf_v1['xh2o(ppm)'][10:],s=3,c='k',label='V1')
ax.scatter(plotdf_v2.index[10:],plotdf_v2['xh2o(ppm)'][10:],s=3,c='r',label='V2')
ax.set_ylabel('XH2O (ppm)',size = labsize)
ax.set_xlabel('Datetime (UTC)',size = labsize)
ax.tick_params(labelsize = labsize)
ax.legend(fontsize=labsize)
ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M %Z', tz = plotdf_v2.index.tz))
ax.set_xlabel(plotdf_v2.index[0].strftime('%b %d, %Y'),size = labsize)
plt.gcf().autofmt_xdate()
plt.show()

In [ ]:
labsize = 18

fig,ax = plt.subplots(figsize = (20,10))
ax.scatter(subtract_diff.index,subtract_diff['xh2o(ppm)'],s=3,c='k')
ax.set_ylabel('V1 - V2 XH2O(ppm) Difference',size = labsize)
ax.set_xlabel('Datetime (UTC)',size = labsize)
ax.tick_params(labelsize = labsize)
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d', tz = plotdf_v2.index.tz))
ax.set_xlabel('Datetime UTC',size = labsize)
plt.gcf().autofmt_xdate()
plt.show()

In [ ]:
summary_df = subtract_diff.describe().drop(['count','25%','50%','75%'])
summary_df
#summary_df.to_csv('/uufs/chpc.utah.edu/common/home/u0890904/LAIR_1/Data/v1_v2_subtraction_stats.csv')

# Retrievals after NaN fix
Comparing retrievals before and after Jacobs fix of the NaNs in wco2_6073 and CH4_5938. 

In [ ]:
#Loading the full dataset can take a while
dt1_str = '2022-05-23 00:00:00' #Start Datetime
dt2_str =  '2024-05-01 00:00:00' #End Datetime
tz = 'US/Mountain' #Timezone to load the data into

new_retrievals_folder = "/uufs/chpc.utah.edu/common/home/u0890904/LAIR_1/Data/EM27_oof/ha_new/"
new_oof_manage = ac.oof_manager(new_retrievals_folder,tz) #setup the oof manager
new_em27_df = new_oof_manage.load_oof_df_inrange(dt1_str,dt2_str,filter_flag_0=True,print_out=True,cols_to_load=None) #load the dataframe for inrange data

old_retrievals_folder = "/uufs/chpc.utah.edu/common/home/u0890904/LAIR_1/Data/EM27_oof/SLC_EM27_ha_2022_2023_oof_v2_nasrin_correct/"
old_oof_manage = ac.oof_manager(old_retrievals_folder,tz) #setup the oof manager
old_em27_df = old_oof_manage.load_oof_df_inrange(dt1_str,dt2_str,filter_flag_0=True,print_out=True,cols_to_load=None) #load the dataframe for inrange data

new_resampled = new_em27_df.resample('5min').mean(numeric_only=True).dropna()
old_resampled = old_em27_df.resample('5min').mean(numeric_only=True).dropna()
new_resampled = new_resampled.loc[(new_resampled.index<old_resampled.index.max())]

In [ ]:
fig = make_subplots(rows=3,cols=1,shared_xaxes=True)

fig.add_trace(go.Scatter(
    x = new_resampled.index,
    y = new_resampled['xch4(ppm)'],
    mode = 'markers',
    marker = {'color' : 'black','size':3},
    name = 'no_nans',
    showlegend = True,
),row = 1,col = 1)

fig.add_trace(go.Scatter(
    x = old_resampled.index,
    y = old_resampled['xch4(ppm)'],
    mode = 'markers',
    marker = {'color' : 'purple','size':3},
    name = 'with_nans',
    showlegend = True,
),row = 1,col = 1)

fig.add_trace(go.Scatter(
    x = new_resampled.index,
    y = new_resampled['xco2(ppm)'],
    mode = 'markers',
    marker = {'color' : 'black','size':3},
    name = 'no_nans',
    showlegend = False,
),row = 2,col = 1)
fig.add_trace(go.Scatter(
    x = old_resampled.index,
    y = old_resampled['xco2(ppm)'],
    mode = 'markers',
    marker = {'color' : 'purple','size':3},
    name = 'with_nans',
    showlegend = False,
),row = 2, col = 1)

fig.add_trace(go.Scatter(
    x = new_resampled.index,
    y = new_resampled['xco(ppb)'],
    mode = 'markers',
    marker = {'color' : 'black','size':3},
    name = 'no_nans',
    showlegend = False,
),row = 3, col = 1)
fig.add_trace(go.Scatter(
    x = old_resampled.index,
    y = old_resampled['xco(ppb)'],
    mode = 'markers',
    marker = {'color' : 'purple','size':3},
    name = 'with_nans',
    showlegend = False,
),row = 3, col = 1)


fig.show()

In [ ]:
subtract_diff = new_resampled[['xch4(ppm)','xco2(ppm)']].subtract(old_resampled[['xch4(ppm)','xco2(ppm)']])

summary_df = subtract_diff.describe().drop(['count','25%','50%','75%'])
sdf = summary_df.loc[:, (summary_df != 0.0).any(axis=0)]
for col in sdf.columns:
    sdf = sdf.rename(columns={col:f'{col}_diff'})
sdf

# UA HA 2024 Comparison

In [ ]:
#ac.copy_em27_oofs_to_singlefolder('/uufs/chpc.utah.edu/common/home/lin-group9/agm/EM27/ha/results','/uufs/chpc.utah.edu/common/home/u0890904/LAIR_1/Data/EM27_oof/ha')

In [ ]:
#Loading the full dataset can take a while
dt1_str = '2024-07-20 05:00:00' #Start Datetime
dt2_str =  '2024-09-10 00:00:00' #End Datetime
tz = 'US/Mountain' #Timezone to load the data into

ua_folder = "/uufs/chpc.utah.edu/common/home/u0890904/LAIR_1/Data/EM27_oof/ua/"
ua_oof_manage = ac.oof_manager(ua_folder,tz) #setup the oof manager
ua_df = ua_oof_manage.load_oof_df_inrange(dt1_str,dt2_str,filter_flag_0=False,print_out=True,cols_to_load=None) #load the dataframe for inrange data
ua_df = ua_df.loc[(ua_df['flag']==0) | (ua_df['flag']==23)]
ua_resampled = ua_df.resample('1min').mean(numeric_only=True)

ha_folder = "/uufs/chpc.utah.edu/common/home/u0890904/LAIR_1/Data/EM27_oof/ha/"
ha_oof_manage = ac.oof_manager(ha_folder,tz) #setup the oof manager
ha_df = ha_oof_manage.load_oof_df_inrange(dt1_str,dt2_str,filter_flag_0=False,print_out=True,cols_to_load=None) #load the dataframe for inrange data
ha_df = ha_df.loc[(ha_df['flag']==0)]
ha_resampled = ha_df.resample('1min').mean(numeric_only=True)

In [ ]:
subdt1 = '2024-07-01 00:00:00'
subdt2 = '2024-09-10 00:00:00'
sub_ha = ha_resampled.loc[(ha_resampled.index>subdt1)&(ha_resampled.index<subdt2)].dropna()
sub_ua = ua_resampled.loc[(ua_resampled.index>subdt1)&(ua_resampled.index<subdt2)].dropna()

params = ['xch4(ppm)','xco2(ppm)','xco(ppb)','xluft','fvsi(%)']
df_dict = {'ha':[sub_ha,'red',5],'ua':[sub_ua,'black',3]}

fig = make_subplots(rows=len(params),cols=1,shared_xaxes=True)

row = 1
for param in params:
    for key,values in df_dict.items():
        df = values[0]
        color = values[1]
        leg = False if row>1 else True
        fig.add_trace(go.Scatter(
        x = df.index,
        y = df[param],
        mode = 'markers',
        marker = {'color' : color,'size':values[2]},
        name = key,
        showlegend = leg,
        ),row = row,col = 1)
        fig.update_yaxes(title_text = param,row=row,col=1)
    row+=1

fig.update_layout(
    height=100*len(params),
    margin = go.layout.Margin(t=1,b=1)
)
fig.show()

In [18]:
ds = xr.open_dataset('/uufs/chpc.utah.edu/common/home/lin-group9/agm/EM27/ha/results/daily/20240729/ha20240729_20240730.private.nc')
fs_list = []
for dv in ds.data_vars:
    if 'fs' in dv:
        fs_list.append(dv)

fs_df = ds[fs_list].to_dataframe()
fs_df.index = fs_df.index.tz_localize('UTC').tz_convert('US/Mountain')

In [ ]:
i = 2
fs_var = fs_list[i]

fig = make_subplots(rows=2,cols=1)

fig.add_trace(go.Scatter(
    x = fs_df.index,
    y = fs_df[fs_var],
    mode = 'markers',
    marker = {'color' : 'black','size':3},
    name = 'ua',
    showlegend = False,
),row = 2,col = 1)
fig.update_yaxes(title_text = 'fvsi(%)',row=1,col=1)
fig.update_yaxes(title_text = fs_var,row=2,col=1)